# <font color="blue"> MBA em Ciência de Dados</font>
# <font color="blue">Técnicas Avançadas para Captura e Tratamento de Dados</font>

## <font color="blue"> Matriz Documento $\times$ Palavras - Bag of Words</font>
    
## <font color="blue">Avaliação Solução</font>

**Material Produzido por Luis Gustavo Nonato**<br>
**Cemeai - ICMC/USP São Carlos**
---

Os exercícios abaixo fazem uso da coleção de documentos presente no diretório `DocCol2` contido no arquivo <font style="font-family: monaco"> DocCol.zip</font>, o qual pode ser baixado do Moodle.

In [1]:
import glob
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
import pandas as pd
import numpy as np
from collections import Counter

### Exercício 1)
Armazene os documentos disponíveis no diretório `DocCol2` em um dicionário onde a chave é o nome do arquivo e o valor é a string contida no arquivo. O documento contendo a menor string é:

a) gr7<br>
<font color='red'> b) au1 </font><br>
c) ch37 <br>
d) au81


In [2]:
folder = "../DocCol2/*" # aqui vc deve mudar para o caminho onde o diretório DocCol2 está
files = glob.glob(folder)

docs = {}
larger_doc = ['',1e10]
for fname in files:
    with open(fname,'r') as f:
        key = fname.split('/')[-1]
        docs[key] = f.read() 
        sizes = len(docs[key])
        if larger_doc[1] > sizes:
            larger_doc[0] = key
            larger_doc[1] = sizes
              
print('documento com a menor string: ',larger_doc[0],'com',larger_doc[1],'caracteres')

documento com a menor string:  au1 com 4573 caracteres


### Exercício 2)
Crie um dicionário chamado `docsXwords` onde as chaves são os nomes dos arquivos e os valores correspondem a lista de palavras do documento correspondente. As palavras em cada uma das listas devem ser constituídas apenas por letras do alfabeto, estarem lexicamente normalizadas e contendo dois ou mais caracteres. Qual o documento com o maior número de palavras associadas no dicionário `docsXwords`:

a) gr22 com 1701 palavras<br>
b) ch30 com 1513 palavras<br>
<font color='red'> c) au2 com 1678 palavras</font><br>
d) au8 com 1525 palavras

In [3]:
docsXwords = {}
for key in docs.keys():
    #strings = docs[key].split(' ')
    words = nltk.word_tokenize(docs[key])
    words = [w.lower() for w in words if w.isalpha() and len(w) != 1]
    words = [PorterStemmer().stem(w) for w in words]
    docsXwords[key] = words
    
d={}
for key,value in docsXwords.items():
    d[key] = len(value)
    
dsorted = sorted(d.items(), key=lambda x: x[1], reverse=True)
print(dsorted[:5])

[('au2', 1678), ('ch30', 1525), ('au14', 1503), ('ch44', 1481), ('ch28', 1422)]


### Exercício 3)
As listas de palavras do dicionário `docsXwords` possuem palavras repetidas. Construa um dicionário chamado `corpus` onde a chave é uma palavra que aparece nas listas de `docsXwords` e os valores correspondem ao número de vezes que a palavra apareceu em `docsXwords`. Por exemplo, supondo que `docsXwords` seja:

```python
'ch3':['the', 'nicen', 'creed']
'au14':['the', 'river', 'creed', 'but']
'gr8':['river', 'the', 'nicen', 'creed']
```
o dicionário `corpus` será:
```python
'the':3
'nicen':2
'creed':3
'river':2
'but':1
```
Quantas palavras estão associadas ao valor 1 no dicionário `corpus`?

a) 2103<br>
b) 1867<br>
<font color='red'> c) 2862 </font><br>
d) 3001

In [6]:
# unificando todas as palavras em uma única lista
corpus = [palavras for sublista in list(docsXwords.values()) for palavras in sublista]

corpus = dict(Counter(corpus))
single_words = list(filter(lambda x: x[1] == 1,corpus.items()))
print('Número de palavras associadas ao valor 1 no corpus:',len(single_words))

Número de palavras associadas ao valor 1 no corpus: 2862


### Exercício 4)
Qual o documento cuja lista de palavras associadas em `docsXwords` possui o **maior** número de "stop words"? Quantas "stop words" aparecem neste documento:

<font color='red'> a) ch30 com 835 "stop words" </font><br>
b) gr17 com 147 "stop words"<br>
c) au2 com 717 "stop words"<br>
d) gr17 com 637 "stop words"

**Dica**: Crie um dicionário a partir de docsXwords onde a chave é o nome do documento e o valor é o número de stop words no documento.

In [7]:
# lista das stop words
nltk.download('stopwords')
stop_words = stopwords.words('english')

d={}
for key,value in docsXwords.items():
    nantes = len(value)
    ndepois = len([w for w in value if w not in stop_words]) 
    d[key] = nantes - ndepois  # número de stop words
    
dsorted = sorted(d.items(), key=lambda x: x[1], reverse=False)
print(dsorted[-5:])

[('ch21', 704), ('ch24', 708), ('au2', 717), ('ch44', 734), ('ch30', 835)]


[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/lgnonato/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


### Exercício 5) 
Construa uma matriz Documentos $\times$ Palavras a partir do dicionário `docsXwords` para a coleção de documentos do diretório `DocCol2`. Qual coluna da matriz possui a menor soma de valores? Qual o valor da soma?

a) 'au1' com soma 3451<br>
b) 'ch31' com soma 1754 <br>
<font color='red'> c) 'gr5' com soma  256</font><br>
d) 'ch3' com soma 125

In [8]:
df_dXp = pd.DataFrame(data=np.zeros((len(corpus.keys()),len(docsXwords.keys()))),\
                  columns=list(docsXwords.keys()),index=list(corpus.keys()))

for key,value in docsXwords.items():
    dtemp = dict(Counter(value))
    df_dXp.loc[list(dtemp.keys()),key] = list(dtemp.values())
    
print(df_dXp.head())

         ch9  ch38  ch31  ch7  ch36  ch37  ch1  ch30  ch6  ch39  ...  ch16  \
from     6.0   3.0   6.0  1.0   9.0   9.0  7.0  41.0  6.0   5.0  ...  10.0   
kempmp   3.0   0.0   0.0  0.0   0.0   0.0  0.0   0.0  0.0   0.0  ...   0.0   
petri    4.0   0.0   0.0  0.0   0.0   0.0  0.0   0.0  0.0   0.0  ...   0.0   
pihko    5.0   0.0   0.0  0.0   0.0   0.0  0.0   0.0  0.0   0.0  ...   0.0   
subject  2.0   1.0   1.0  1.0   1.0   1.0  3.0   1.0  1.0   2.0  ...   1.0   

         ch11  ch43  ch44  ch10  ch28  ch17  ch21  ch19  ch26  
from      1.0   5.0   5.0   3.0  10.0   7.0   6.0   5.0   5.0  
kempmp    0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   1.0  
petri     0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   2.0  
pihko     0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   3.0  
subject   1.0   1.0   1.0   1.0   2.0   2.0   1.0   1.0   2.0  

[5 rows x 91 columns]


In [13]:
S = df_dXp.sum(axis=0)
# S_sorted = S.sort_values(ascending=True)
# S_sorted.head()
print('coluna com menor soma',S.idxmin(),'com valor',S.min())

coluna com menor soma gr5 com valor 256.0
